In [1]:
import pandas as pd

In [2]:
user_item_data = pd.read_pickle('user_item_data.pickle')

In [3]:
import numpy as np

unique_values = user_item_data['user_id'].unique()
num_chunks = 300
value_chunks = np.array_split(unique_values, num_chunks)

user_item_matrices = []

for chunk_values in value_chunks:
    chunk_data = user_item_data[user_item_data['user_id'].isin(chunk_values)]    
    user_item_matrix_chunk = chunk_data.pivot_table(index='user_id', columns='gmap_id', values='rating', fill_value=0)

    user_item_matrices.append(user_item_matrix_chunk)

In [4]:
user_item_matrices[0]

gmap_id,0x145e90b5d53fcc9d:0x2ae4bdf777e04ee3,0x145e9b56da9601ad:0x1dbdccbfc0ec8184,0x145ec14420418d4f:0x204b57b935cbab8b,0x80c299fafee52d97:0x9da4a27da8b056d4,0x86342a851b3c61a5:0xee213f2c78862880,0x863631bee71aa6cb:0xa55be3711931a5b5,0x8640948510b69399:0xf81cf490f925c6de,0x8640948788c6d617:0x65b6f95554494bf3,0x864094897064750f:0xaddec86a3b32702d,0x8640950b2e7ad955:0xebefacaaafb01099,...,0x865cf79c27e99d83:0x4c86e03cb946f412,0x8665a0a1c17a067f:0xd9d269d91e1ba049,0x86680100299e0aa5:0x18d70c011361f62d,0x866f8360dea86c3d:0x4b1305d9105d36f1,0x866f836123ad9f7f:0x59b801e3c21245bf,0x866f836175074b39:0x5c6f78bb88727b83,0x86fdebb996524a09:0x1b05143868b42464,0x86fdebc78c248a19:0xbfa5b8a72f79d25c,0x86ffd55468893f2f:0x2e90124791cac33,0x87b21c41ae9172a7:0xea3bdd828b6e030c
user_id,,,,,,,,,,,,,,,,,,,,,
100000008184337496392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000025795212501306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000073667291247647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000087261232833762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000097188632545638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100060586486635620045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100060643407503221384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100060658756506484334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Initialize an empty set to store unique GMap IDs
unique_gmap_ids = set()

# Iterate through each user-item matrix
for matrix in user_item_matrices:
    # Extract the unique GMap IDs from the matrix
    gmap_ids = set(matrix.columns)  # Assuming the GMap IDs are column names
    # Update the set of unique GMap IDs
    unique_gmap_ids.update(gmap_ids)

# Convert the set of unique GMap IDs to a list (if needed)
unique_gmap_ids_list = list(unique_gmap_ids)

In [6]:
import numpy as np

filled_matrices = []

for matrix in user_item_matrices:
    filled_matrix = np.zeros((matrix.shape[0], len(unique_gmap_ids_list)))
    for idx, gmap_id in enumerate(unique_gmap_ids_list):
        if gmap_id in matrix.columns:
            # print(gmap_id)
            filled_matrix[:, idx] = matrix[gmap_id].values 
    filled_matrices.append(filled_matrix)


In [7]:
def calculate_average_rating_pearson(matrix):
    average_rating=[]
    #calculating the number of non zero ratings a user has given
    non_zero_ratings = np.sum(matrix != 0 , axis = 1)

    users,movies = matrix.shape
    #Calculating the average rating per each user
    for i in range(users):
        #calcuating the sum if the user rating
        average_rating.append(matrix[i,:].sum())
        #checking if the non zero ratings are equal to zero or not.
        if non_zero_ratings[i]!= 0 :
            #Allocating user average rating as average of the non zero ratings.
            average_rating[i]= average_rating[i]/non_zero_ratings[i]
        else:
            #Allocating user average rating as zero as there are no non zero ratings.
            average_rating[i] = average_rating[i]/(1e-8)
        
    return average_rating

def calculate_pearson_similarity(matrix):

    # print(matrix)
    n, m = matrix.shape
    average_rating = calculate_average_rating_pearson(matrix)
    centered_matrix = matrix.copy()
    for i in range(n):
        centered_matrix[i,:] -= average_rating[i]
    covergence_matrix = np.dot(centered_matrix,centered_matrix.T)
    squared_centered_matrix = np.square(centered_matrix)

    normalization = []
    for row in squared_centered_matrix:
        normalization.append(np.sum(row))    
    denominator = np.outer(normalization, normalization)
    pearson_matrix = (covergence_matrix)/((np.sqrt(denominator+ 1e-8)) )

    np.fill_diagonal(pearson_matrix, -np.inf)  

    return pearson_matrix


In [8]:
item_item_similarity_matrix = np.zeros((len(unique_gmap_ids_list),len(unique_gmap_ids_list)))

for user_item_matrix_chunk in filled_matrices:
    pearson_matrix = calculate_pearson_similarity(user_item_matrix_chunk.T)
    item_item_similarity_matrix += pearson_matrix

In [9]:
item_item_similarity_matrix = item_item_similarity_matrix/300

In [10]:
item_item_similarity_matrix

array([[      -inf, 0.47916004, 0.61215056, ..., 0.1364487 , 0.03327804,
        0.82485003],
       [0.47916004,       -inf, 0.29634522, ..., 0.08658999, 0.01332005,
        0.41604626],
       [0.61215056, 0.29634522,       -inf, ..., 0.10323029, 0.01997654,
        0.52584535],
       ...,
       [0.1364487 , 0.08658999, 0.10323029, ...,       -inf, 0.        ,
        0.11982428],
       [0.03327804, 0.01332005, 0.01997654, ..., 0.        ,       -inf,
        0.03328623],
       [0.82485003, 0.41604626, 0.52584535, ..., 0.11982428, 0.03328623,
              -inf]])

In [11]:
data_frame = pd.DataFrame(item_item_similarity_matrix, index=unique_gmap_ids_list, columns=unique_gmap_ids_list)

In [12]:
data_frame.to_csv('item_item_rating_similarity_matrix.csv')